# Examining live trading strategy using notebook

In this notebook, we show how to examine live trading strategy instance using Python and Jupyter notebook.

- Downloading the trades from the live instance
- Reading data to `State` object
- Using the same analytics functions we use in backtests against the live data

## Download live data

- Each strategy Docker instances offers a webhook that allows access the data of this strategy, include `State` object
  that is the flat file state of the all the strategy current and past decision making
- Note that `State` class and any children classes in the state tree may have changes  between
  versions and decoding might need you to use a specific version of `trade-executor`


In [1]:
from tradeexecutor.monkeypatch.dataclasses_json import patch_dataclasses_json
from tradeexecutor.state.state import State
import requests

# Currently needed because unpatched dataclasses_json package issues
patch_dataclasses_json()

# Public internet endpoint as exposed by the trade executor Docker
webbhook_url = "https://pancake-eth-usd-sma.tradingstrategy.ai"

state_api = f"{webbhook_url}/state"

resp = requests.get(state_api)

state_blob = resp.content

print(f"Downloaded {len(state_blob):,} bytes state data")

state = State.from_json(state_blob)

print(f"trade-executor was launched at: {state.created_at}, we have {len(list(state.portfolio.get_all_trades()))} trades")

Downloaded 236,228 bytes state data
trade-executor was launched at: 2022-12-01 22:17:08.835950, we have 14 trades


## Analyse the live strategy trades

We can use the same functions as in our backtesting notebooks to produce summaries, graphs.

In [2]:
import pandas as pd
from IPython.core.display_functions import display
from tradeexecutor.analysis.trade_analyser import build_trade_analysis

analysis = build_trade_analysis(state.portfolio)

summary = analysis.calculate_summary_statistics()

with pd.option_context("display.max_row", None):
    display(summary.to_dataframe())

,0
Trading period length,5 days
Return %,-1.81%
Annualised return %,-124.87%
Cash at start,$498.78
Value at end,$489.75
Trade win percent,14.29%
Total trades done,7
Won trades,1
Lost trades,6
Stop losses triggered,4


In [3]:
print("All done")

All done
